# Fundamentos de RDD

#### [Introducción a Spark con Python, por Jose A. Dianes](https://github.com/jadianes/spark-py-notebooks)

Este notebook introducirá tres operaciones básicas pero esenciales de Spark. Dos de ellas son las *transformaciones* `map` y `filter`. La otra es la *acción* `collect`. Al mismo tiempo, introduciremos el concepto de *persistencia* en Spark.

## SparkSession

In [ ]:
# Importamos pyspark y SparkSession
import pyspark
from pyspark.sql import SparkSession

# Nota: solamente hay una SparkSession activa por aplicación
# spark = SparkSession.builder.appName("RDDBasics").getOrCreate()

# Obtenemos el SparkContext desde la SparkSession existente
sc = spark.sparkContext

## Obteniendo los datos y creando el RDD

Como hicimos en nuestro primer notebook, usaremos el conjunto de datos reducido (10 por ciento) proporcionado para la KDD Cup 1999, que contiene casi medio millón de interacciones de red. El archivo se proporciona como un archivo Gzip que descargaremos localmente.

In [ ]:
# Definimos la URL del archivo de datos
url = "http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz"

# Importamos SparkFiles para gestionar la descarga y acceso a archivos
from pyspark import SparkFiles

# Añadimos el archivo al contexto de Spark
spark.sparkContext.addFile(url)

Ahora podemos usar este archivo para crear nuestro RDD.

In [ ]:
# Creamos el RDD leyendo el archivo de texto comprimido
myRDD = sc.textFile("file://" + SparkFiles.get("kddcup.data_10_percent.gz"))

## La transformación `filter`

Esta transformación se puede aplicar a los RDDs para mantener solo los elementos que satisfacen una cierta condición. Más concretamente, se evalúa una función en cada elemento del RDD original. El nuevo RDD resultante contendrá solo aquellos elementos que hagan que la función devuelva `True`.

Por ejemplo, imaginemos que queremos contar cuántas interacciones `normal.` tenemos en nuestro conjunto de datos. Podemos filtrar nuestro RDD `raw_data` de la siguiente manera.

In [ ]:
# Filtramos el RDD para mantener solo las líneas que contienen "normal."
# lambda x: "normal." in x es una función anónima que devuelve True si "normal." está en x
# Esto es una TRANSFORMACIÓN (lazy), no se ejecuta hasta que haya una acción
normal_myRDD = myRDD.filter(lambda x: "normal." in x)

Ahora podemos contar cuántos elementos tenemos en el nuevo RDD.

In [ ]:
# Contamos los elementos filtrados
# count() es una ACCIÓN, aquí es cuando realmente se ejecuta el cálculo
normal_myRDD.count()

In [ ]:
# Contamos el total de elementos en el RDD original para comparar
myRDD.count()

In [ ]:
# Importamos time para medir el tiempo de ejecución
from time import time

# Marcamos el tiempo inicial
t0 = time()

# Ejecutamos la acción count()
normal_count = normal_myRDD.count()

# Calculamos el tiempo transcurrido
tt = time() - t0

# Mostramos los resultados
print("There are {} 'normal' intercations".format(normal_count))
print("Count completed in {} seconds".format(round(tt,3)))

Recuerda del notebook 1 que tenemos un total de 494021 en nuestro conjunto de datos del 10 por ciento. Aquí podemos ver que 97278 contienen la etiqueta `normal.`

Observa que hemos medido el tiempo transcurrido para contar los elementos en el RDD. Lo hemos hecho porque queríamos señalar que los cálculos (distribuidos) reales en Spark ocurren cuando ejecutamos *acciones* y no *transformaciones*. En este caso, `count` es la acción que ejecutamos sobre el RDD. Podemos aplicar tantas transformaciones como queramos en nuestro RDD y no se realizará ningún cálculo hasta que llamemos a la primera acción que, en este caso, tarda unos segundos en completarse.

## La transformación `map`

Al usar la transformación `map` en Spark, podemos aplicar una función a cada elemento en nuestro RDD. Las funciones lambda de Python son especialmente expresivas para esto.

En este caso, queremos leer nuestro archivo de datos como un archivo formateado en CSV. Podemos hacer esto aplicando una función lambda a cada elemento en el RDD de la siguiente manera.

In [ ]:
# Visualizamos algunas líneas del RDD original (son strings)
myRDD.take(5)

In [ ]:
# Aplicamos la transformación map para dividir cada línea por comas
# lambda x: x.split(",") convierte cada string en una lista
# Esto es una TRANSFORMACIÓN (lazy), no se ejecuta hasta que haya una acción
csv_data = myRDD.map(lambda x: x.split(","))

# Marcamos el tiempo inicial
t0 = time()

# take(5) es una ACCIÓN, aquí es cuando se ejecuta la transformación map
head_rows = csv_data.take(5)

# Calculamos el tiempo transcurrido
tt = time() - t0

print("Parse completed in {} seconds".format(round(tt,3)))

In [ ]:
# Visualizamos las primeras 3 filas parseadas (ahora son listas)
head_rows[0:3]

Nuevamente, toda la acción ocurre una vez que llamamos a la primera *acción* de Spark (es decir, *take* en este caso). ¿Qué pasa si tomamos muchos elementos en lugar de solo los primeros?

In [ ]:
# Tomamos muchos más elementos para ver la diferencia en tiempo de ejecución
t0 = time()

# Solicitamos 100,000 elementos parseados
head_rows = csv_data.take(100000)

tt = time() - t0
print("parse completed in {} seconds".format(round(tt, 3)))

Podemos ver que tarda más. La función `map` se aplica ahora de forma distribuida a muchos elementos del RDD, de ahí el mayor tiempo de ejecución.

### Usando `map` y funciones predefinidas

Por supuesto, podemos usar funciones predefinidas con `map`. Imaginemos que queremos tener cada elemento en el RDD como un par clave-valor donde la clave es la etiqueta (ej. *normal*) y el valor es toda la lista de elementos que representa la fila en el archivo formateado en CSV. Podríamos proceder de la siguiente manera.

In [ ]:
# Visualizamos las líneas originales
myRDD.take(5)

In [ ]:
# Definimos una función personalizada para parsear y crear pares clave-valor
def parse_interaction(line):
    # Dividimos la línea por comas
    elems = line.split(",")
    
    # Extraemos la etiqueta (última columna, posición 41)
    tag = elems[41]
    
    # Devolvemos una tupla (clave, valor)
    # Clave: la etiqueta (normal., attack., etc.)
    # Valor: la lista completa de elementos
    return (tag, elems)

# Aplicamos la función personalizada con map
key_csv_data = myRDD.map(parse_interaction)

# Tomamos los primeros 5 elementos y mostramos los primeros 2
head_rows = key_csv_data.take(5)
print(head_rows[0:2])

Eso fue fácil, ¿verdad?

En nuestro notebook sobre el trabajo con pares clave-valor, usaremos este tipo de RDDs para hacer agregaciones de datos (ej. contar por clave).

## La acción `collect`

Hasta ahora hemos usado las acciones `count` y `take`. Otra acción básica que necesitamos aprender es `collect`. Básicamente, traerá todos los elementos del RDD **a la memoria** para que trabajemos con ellos. Por esta razón, debe usarse con cuidado, especialmente cuando se trabaja con RDDs grandes.

Un ejemplo usando nuestros datos crudos.

In [ ]:
# Marcamos el tiempo inicial
t0 = time()

# collect() trae TODOS los datos del RDD al nodo driver (memoria local)
# ¡PRECAUCIÓN! Puede causar problemas de memoria con datasets grandes
all_raw_data = myRDD.collect()

tt = time() - t0
print("Data collected in {} seconds".format(round(tt,3)))

In [ ]:
# Ahora all_raw_data es una lista de Python normal
# Podemos acceder a los elementos usando indexación estándar
all_raw_data[:3]

Eso tardó más que cualquier otra acción que usamos antes, por supuesto. Cada nodo trabajador de Spark que tiene un fragmento del RDD debe ser coordinado para recuperar su parte, y luego *reducir* todo junto.

Como último ejemplo que combina todo lo anterior, queremos recopilar todas las interacciones `normal` como pares clave-valor.

In [ ]:
# Paso 1: Parseamos en pares clave-valor usando la función definida anteriormente
key_csv_data = myRDD.map(parse_interaction)

# Paso 2: Filtramos solo las interacciones con clave "normal."
# x[0] es la clave (la etiqueta), x[1] sería el valor (la lista completa)
normal_key_interactions = key_csv_data.filter(lambda x: x[0] == "normal.")

# Paso 3: Recopilamos todos los datos filtrados
t0 = time()
all_normal = normal_key_interactions.collect()
tt = time() - t0

# Contamos los elementos en la lista resultante
normal_count = len(all_normal)

print("Data collected in  {} seconds".format(round(tt, 3)))
print("There are {} 'normal' interactions".format(normal_count))

In [ ]:
# Verificamos el número de particiones del RDD original
print("Number of Partitions: " + str(myRDD.getNumPartitions()))

# Verificamos el número de particiones después de las transformaciones
# Las transformaciones mantienen el número de particiones por defecto
print("Number of Partitions: " + str(key_csv_data.getNumPartitions()))

In [ ]:
# Verificamos el tipo de objeto RDD
type(myRDD)

Este conteo coincide con el conteo anterior de interacciones `normal`. El nuevo procedimiento consume más tiempo. Esto se debe a que recuperamos todos los datos con `collect` y luego usamos la función `len` de Python en la lista resultante. Antes solo estábamos contando el número total de elementos en el RDD usando `count`.